# Raport z rozwiązania problemu

W danych znalazłem nastepujące "problemy":
1. Linija zawierająca tylko i wyłącznie cudzysłów (i tabulator po niej).
2. Niektóre tweety (cały tekst) są objęte w cudzysłów, a niektóre nie.
3. Podwójne cudzosłowy oznaczające tak naprawdę cudzysłowy pojedyncze.

Rozwiązanie:
1. zakładam, że jest to sytuacja, która ogólnie może się zdarzyć i polega na tym, że cudzysłów z wiadomości w danej lini może znaleźć się w następnej lini. Można by to uogólnić na przypadki, gdy w wiadomości jest znak przejścia do nowej lini i część wiadomości przenosi się do kolejnej lini, ale uznaję, że nie ma takiej potrzeby.
2. Zakładam, że jest to niepoążadna cech wynikła być może z zbierania tweetów dwoma różnymi sposobami. Ujednolicam tweety, usuwając cudzysłowy z początku i końca w tweetach, wkótrych wsytępuje znak '"' jednocześnie na końcu i początku. Nie jest to podejście doskonałe, bo może się zdarzyć, że po te cuchysłowy obejmują słowa/zwroty z początku i końca tweeta, ale z praktycznego punktu widzenia, uznaję takie podejście za właściwe.
3. Zamieniam podwójne cudzysłowy na pojedyczne ('""' -> '"').


In [1]:
from tools import *
from models import *

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


### Dane:

In [2]:
train = "data/twitter-2016train-A.txt"
dev = "data/twitter-2016dev-A.txt"
test = "data/twitter-2016test-A.txt"

x_train, y_train = read_data(train)
x_dev, y_dev = read_data(dev)
x_test, y_test = read_data(test)

Data report:
 Number of obserwations:  6000
Data report:
 Number of obserwations:  1999
Data report:
 Number of obserwations:  20632


In [3]:
x_train[0]

"dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon."

### Transformacja danych

In [4]:
x_train = preprocess(x_train)
x_dev = preprocess(x_dev)
x_test = preprocess(x_test)

In [5]:
x_train[0]

"dear _user_ the newooffice for mac is great and all, but no lync update? c'mon."

In [7]:
#wymiar embeddingow jaki chcemy uzyc i sciezka do pliku z embeddingami

# aby uruchomic, nalezy umiescic w folderze "data/glove.twitter.27B" plik z odpowiednimi embeddingami:
# https://nlp.stanford.edu/projects/glove/

emb_dim = 25

embedding_path = "data/glove.twitter.27B/glove.twitter.27B." + str(emb_dim) + "d.txt"

words2ids, embeddings = load_embeddings(embedding_path, emb_dim)

### Przetworzenie danych do postaci wymaganej przez keras

In [9]:
maxlen = max([len(x) for x in x_train] + [len(x) for x in x_dev] + [len(x) for x in x_test])

x_train, y_train = transform(x_train, y_train, words2ids, maxlen)
x_dev, y_dev = transform(x_dev, y_dev, words2ids, maxlen)
x_test, y_test= transform(x_test, y_test, words2ids, maxlen)

In [10]:
x_train[0]

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [11]:
# w tej chwili testujemy kod na malutkich podzbiorach

x_train = x_train[:75]
y_train = y_train[:75]

x_dev = x_dev[:25]
y_dev = y_dev[:25]

x_test = x_test[:100]
y_test = y_test[:100]

### Uczenie i testowanie modeli

In [12]:
vocabulary_dim = len(words2ids)

names = ["LSTM", "BidirectionalLSTM"]
models = [LSTMmodel(100, embeddings, vocabulary_dim, emb_dim, maxlen), 
          BidirectionalLSTMmodel(100, embeddings, vocabulary_dim, emb_dim, maxlen)]



results = []

for model,name in zip(models,names):

    print("Model : ",name)
    
    print(model.summary())
    
    early_stopping = EarlyStopping(patience=3,monitor="val_loss")

    model.fit(x_train, y_train,
              batch_size=32,
              epochs = 30,
              callbacks=[early_stopping],
              validation_data=(x_dev,y_dev))

    results.append((name,model.evaluate(x_test, y_test, verbose=0)[1]))

/usr/local/lib/python3.6/dist-packages/keras/layers/recurrent.py:2024: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '
/usr/local/lib/python3.6/dist-packages/keras/layers/recurrent.py:2024: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


Model :  LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 362, 25)           29837825  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               50400     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 29,888,528
Trainable params: 29,888,528
Non-trainable params: 0
_________________________________________________________________
None
Train on 75 samples, validate on 25 samples
Epoch 1/1
75/75 [==============================] - 12s 165ms/step - loss: 1.0973 - categorical_accuracy: 0.3733 - val_loss: 1.0925 - val_categorical_accuracy: 0.3600
Model :  BidirectionalLSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedd

### Wyniki [w tej chwili sa to wyniki bezwartosciowe, bo to tylko sprawdzenie czy kod dziala]

In [14]:
results = pd.DataFrame(results)

In [15]:
results

,0,1
0,LSTM,0.156656
1,BidirectionalLSTM,0.156705
